# How to make the .exe file

https://www.marinamele.com/from-a-python-script-to-a-portable-mac-application-with-py2app

In the terminal:

In [ ]:
pip install -U py2app
py2applet --make-setup main.py

Modify the setup file:

In [ ]:


"""
This is a setup.py script generated by py2applet

Usage:
    python setup.py py2app
"""

from setuptools import setup

APP = ['main.py']
DATA_FILES = ['icons/Menu_icon.png', 'icons/App_icon.icns', 'rump.py', '.env', 'chatbox_WEB.py', 'enter_api_WEB.py', 'langchain_functions.py']
OPTIONS = {
    'argv_emulation': True,
    'plist': {
        'CFBundleName': 'Majid',
        'CFBundleDisplayName': 'Majid',
        'CFBundleIconFile': 'App_icon.icns'  # <- this tells macOS which icon to use
    },
    'packages': [],
}
setup(
    app=APP,
    data_files=DATA_FILES,
    options={'py2app': OPTIONS},
    setup_requires=['py2app'],
)

Test if it works (only for testing):

In [ ]:
python setup.py py2app -A

Then create the .exe file:

In [ ]:
python setup.py py2app

# Writable vs. read-only folders

All the files and data will be stored in a base folder. Therefore, we need to set the base folder for all external .py or data:

In [ ]:

        if getattr(sys, "_MEIPASS", False):
            base_path = sys._MEIPASS
        else:
            base_path = os.path.dirname(os.path.abspath(__file__))

        appIcon=os.path.join(base_path, "icons", "Menu_icon.png")

However, this folder is read-only!

If we have files like .env that will be over written, they need to be in another document that is writable:

In [ ]:
user_dir = os.path.expanduser("~/Library/Application Support/Majid")
os.makedirs(user_dir, exist_ok=True)
env_path = os.path.join(user_dir, ".env")

# Load .env (create if missing)
if not os.path.exists(env_path):
    with open(env_path, "w") as f:
        f.write("")  # empty file placeholder
load_dotenv(dotenv_path=env_path)

# Kill web pages

If we use flask API and web based GUI, the ports remain occcupied. So if you see an error persisting, perhaps need to free the port first:

In [ ]:

def free_port(port):
    try:
        result = subprocess.run(
            ["lsof", "-ti", f"tcp:{port}"],
            capture_output=True,
            text=True
        )
        pid = result.stdout.strip()
        if pid:
            subprocess.run(["kill", "-9", pid])
            print(f"Killed process {pid} using port {port}")
        else:
            print(f"No process found on port {port}")
    except Exception as e:
        print("Error freeing port:", e)

#--------------------------------------------------
# Run server
#--------------------------------------------------
if __name__ == "__main__":
    free_port(5006)
    webbrowser.open("http://127.0.0.1:5006")
    app.run(port=5006, debug=False)